In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd

In [10]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
name_dict = {'assessments': 'https://disk.yandex.ru/d/iSVP1sZ7AdW7TQ',
             'courses': 'https://disk.yandex.ru/d/jmJxR8yhwCXlaw',
             'student_assessment': 'https://disk.yandex.ru/d/DYd4M5b_8RETsA',
             'student_registration': 'https://disk.yandex.ru/d/lqHnH8NOZ458iw'}
# Получаем загрузочную ссылку
for key, value in name_dict.items():
    final_url = base_url + urlencode(dict(public_key=value))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # Загружаем файл и сохраняем его
    download_response = requests.get(download_url)
    with open(f'{key}.csv', 'wb') as f:   # Здесь укажите нужный путь к файлу
        f.write(download_response.content)

assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
student_assessment = pd.read_csv('student_assessment.csv')
student_registration = pd.read_csv('student_registration.csv')

1) assessments.csv — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).

code_module — идентификационный код предмета.

code_presentation — семестр (Идентификационный код).

id_assessment — тест (Идентификационный номер ассессмента).

assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).

date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).

weight — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

2) courses.csv — файл содержит список предметов по семестрам.

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код).

module_presentation_length — продолжительность семестра в днях.

3) studentAssessment.csv — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.

id_assessment — тест (идентификационный номер).

id_student — идентификационный номер студента.

date_submitted — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.

is_banked — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).

score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

4) studentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.

code_module — предмет (идентификационный код).

code_presentation — семестр (идентификационный код)

id_student — идентификационный номер студента.

date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

### 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

In [11]:
# Связь фреймов данных о студентах и их результатах по тестам
assessment_df = student_assessment.merge(assessments, how='left', on='id_assessment')

In [12]:
# Число студентов, которые успешно сдали только один курс
assessment_df.query('score >= 40 and assessment_type == "Exam"') \
             .groupby('id_student', as_index=False) \
             .agg({'id_assessment': 'count'}) \
             .rename(columns={'id_assessment': 'num_of_passed_exams'}) \
             .query('num_of_passed_exams == 1').shape[0]

3802

### 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов) 
завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [13]:
# Студенты, которые сдавали экзамены с группировкой по предмету, семестру и номеру экзамена
all_students = assessment_df.query('assessment_type == "Exam"') \
                            .groupby(['code_module', 'code_presentation', 'id_assessment'], as_index=False) \
                            .agg({"id_student": 'count'}) \
                            .rename(columns={'id_student': 'number_of_tried'})

# Студенты, которые успешно сдали экзамены с группировкой по предмету, семестру и номеру экзамена
passed_students = assessment_df.query('assessment_type == "Exam" and score >= 40') \
                               .groupby(['code_module', 'code_presentation', 'id_assessment'], as_index=False) \
                               .agg({"id_student": 'count'}) \
                               .rename(columns={'id_student': 'number_of_passed'})

In [14]:
pd.set_option('display.precision', 3)  # Устанавливаем число знаков после запятой для отображения в датафрейме

In [15]:
# объединяем два фрейма студентов по предмету и создаем колонку завершаемости предмета
exams_df = passed_students.merge(all_students, how='inner')

exams_df['passing_rate'] = exams_df.number_of_passed / exams_df.number_of_tried

exams_df

,code_module,code_presentation,id_assessment,number_of_passed,number_of_tried,passing_rate
0,CCC,2014B,24290,664,747,0.889
1,CCC,2014J,24299,1019,1168,0.872
2,DDD,2013B,25340,504,602,0.837
3,DDD,2013J,25354,878,968,0.907
4,DDD,2014B,25361,485,524,0.926
5,DDD,2014J,25368,842,950,0.886


In [16]:
# определяем минимальное и максимальное значения завершаемости курсов
min_ending = exams_df.passing_rate.min()

max_ending = exams_df.passing_rate.max()

In [17]:
# Экзамен, код экзамена и номер теста с самой низкой завершаемостью
exams_df.query('passing_rate == @min_ending')[['code_module', 'code_presentation', 'id_assessment']]

,code_module,code_presentation,id_assessment
2,DDD,2013B,25340


In [18]:
# Экзамен, код экзамена и номер теста  с самой высокой завершаемостью
exams_df.query('passing_rate == @max_ending')[['code_module', 'code_presentation', 'id_assessment']]

,code_module,code_presentation,id_assessment
4,DDD,2014B,25361


### 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

In [19]:
# Определяем средний срок сдачи экзамена
mean_pass_date_df = assessment_df.query('assessment_type == "Exam" and score >= 40') \
                                 .groupby(['code_module', 'code_presentation'], as_index=False) \
                                 .agg({"date_submitted": 'mean'}) \
                                 .rename(columns={'date_submitted': 'average_pass_date'})

mean_pass_date_df

,code_module,code_presentation,average_pass_date
0,CCC,2014B,231.581
1,CCC,2014J,244.404
2,DDD,2013B,230.165
3,DDD,2013J,239.509
4,DDD,2014B,234.936
5,DDD,2014J,242.804


### 4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

In [20]:
# ТОП-3 курса по числу регистраций, учитываем, что студент мог зарегестрироваться на курс несколько раз (nunique)
reg_df = student_registration.groupby('code_module', as_index=False) \
                             .agg({'id_student': 'nunique'}) \
                             .rename(columns={'id_student': 'number_of_reg'}) \
                             .sort_values('number_of_reg', ascending=False)
reg_df.head(3)

,code_module,number_of_reg
1,BBB,7692
5,FFF,7397
3,DDD,5848


In [21]:
# ТОП-3 курса по оттоку студентов, берем в расчет только студентов,
# у которых имеется факт отмены регистрации на курс (query по date_unregistration)
unreg_df = student_registration.query('date_unregistration.isnull() == False', engine='python') \
                               .groupby('code_module', as_index=False) \
                               .agg({'id_student': 'nunique'}) \
                               .rename(columns={'id_student': 'number_of_unreg'}) \
                               .sort_values('number_of_unreg', ascending=False)

unreg_df.head(3)

,code_module,number_of_unreg
1,BBB,2314
5,FFF,2249
3,DDD,2065


В ТОП-3 по числу регистраций и оттоку попали одинаковые курсы, проверим коэффициент оттока студентов по всем курсам:


In [22]:
churn_df = unreg_df.merge(reg_df, how='inner')

churn_df['churn_rate'] = churn_df.number_of_unreg / churn_df.number_of_reg

churn_df.sort_values('churn_rate', ascending=False)

,code_module,number_of_unreg,number_of_reg,churn_rate
3,CCC,1858,4251,0.437
2,DDD,2065,5848,0.353
1,FFF,2249,7397,0.304
0,BBB,2314,7692,0.301
4,EEE,693,2859,0.242
6,AAA,116,712,0.163
5,GGG,287,2525,0.114


Таким образом, курсы, попавшие в ТОП-3 по числу регистраций и отмен, не возглавляют Таблицу по коэффиценту оттока

### Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. (10 баллов) 

In [23]:
def Cohort():
    '''Makes dataframe for Cohort analysis.
    
       Calculate passing_rate and average_pass_date
       for each code_presentation,
       using code_presentation as index'''
    # импортируем датафреймы
    assessments = pd.read_csv('assessments.csv')
    # создаем датафреймы для анализа
    assessment_df = student_assessment.merge(assessments, how='left', on='id_assessment')
    # число студентов, сдававших экзамены
    all_students = assessment_df.query('assessment_type == "Exam"') \
                                .groupby('code_presentation', as_index=False) \
                                .agg({"id_student": 'count'}) \
                                .rename(columns={'id_student': 'number_of_tried'})
    # студенты, успешно сдавшие экзамены
    passed_students = assessment_df.query('assessment_type == "Exam" and score >= 40') \
                                   .groupby('code_presentation', as_index=False) \
                                   .agg({"id_student": 'count'}) \
                                   .rename(columns={'id_student': 'number_of_passed'})
    # датафрейм для расчета завершаемости
    exams_df = passed_students.merge(all_students, how='inner')
    exams_df['passing_rate'] = exams_df.number_of_passed / exams_df.number_of_tried
    # датафрейм среднего времени сдачи экзаменов за семестр
    average_pass_date = assessment_df.query('assessment_type == "Exam" and score >= 40') \
                                     .groupby('code_presentation', as_index=False) \
                                     .agg({"date_submitted": 'mean'}) \
                                     .rename(columns={'date_submitted': 'average_pass_date'})
    # Объединяем датафрейм с завершаемостью курсов на семестре и средним временем завершения экзамена
    df = exams_df.merge(average_pass_date, how='inner')
    df = df.set_index('code_presentation')
    return df

In [36]:
# Создаем датафрейм с помощью функции
df = Cohort()

In [26]:
# семестр с самой низкой завершаемостью курсов
df['passing_rate'].idxmin()

'2013B'

In [27]:
# семестр c самыми долгими средними сроками сдачи курсов
df['average_pass_date'].idxmax()

'2014J'

## 6. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. Описание подхода можно найти тут. (35 баллов)

In [28]:
# создадим датафрейм для кластеризации:
# датафрейм со средним количеством баллов, средним временем сдачи экзамена и числом попыток сдать экзамен
# с группировкой по студентам
passing_students = assessment_df.query('assessment_type == "Exam"') \
                                .groupby('id_student', as_index=False) \
                                .agg({"score": "mean", "date_submitted": 'mean', "code_module": 'count'}) \
                                .rename(columns={'code_module': 'number_of_tried', "score": 'average_score',
                                                 'date_submitted': 'average_pass_date'}) \
                                .sort_values('number_of_tried', ascending=False)
# датафрейм с числом сданных экзаменов в группировке по студентам
passed_students = assessment_df.query('assessment_type == "Exam" and score >= 40') \
                               .groupby('id_student', as_index=False) \
                               .agg({"code_module": 'count'}) \
                               .rename(columns={'code_module': 'number_of_passed'}) \
                               .sort_values('number_of_passed', ascending=False)
# общий датафрейм издвух предыдущих
students = passing_students.merge(passed_students, how='left', on='id_student')
# заполняем пустые значения для студентов, не сдавших экзамен
students.fillna(0, inplace=True)
# Колонка завершаемости курсов
students['passing_rate'] = students.number_of_passed / students.number_of_tried

In [29]:
# Принцип кластерного анализа заключается в определении подмножеств некоторого объема данных, 
# разделенных определенными показателями с рангами, в нашем случае рангами будут выступать:
# R-среднее время сдачи одного экзамена, F - завершаемость курсов и M - среднее количество
# баллов, получаемое за экзамен, ранг 1 будет являться наилучшим, 3-наихудшим.
# Таким образом, можно получить 27 групп различных конфигураций, что
# способствует более глубокому анализу
# Подготавливаем данные для кластеризации
RFM = students.drop(columns=['number_of_passed', 'number_of_tried'])

RFM = RFM.set_index('id_student')

RFM = RFM.reindex(columns=['average_pass_date', 'passing_rate', 'average_score'])

In [30]:
# Сегментирование проводим по 3 группам, для этого определим границы групп:
quantiles = RFM[['average_pass_date', 'passing_rate', 'average_score']].quantile([.33, .66]).to_dict()

quantiles

{'average_pass_date': {0.33: 236.0, 0.66: 243.0},
 'passing_rate': {0.33: 1.0, 0.66: 1.0},
 'average_score': {0.33: 56.0, 0.66: 76.0}}

In [31]:
# определим принцип ранжирования показателей
# так как среднее количество баллов и завершаемость тем лучше, чем выше значение показателя,
# ранг 1 будет соответствовать максимальному значению,
# а среднее время сдачи экзамена наоборот, т.е. чем выше показатель, тем ниже ранг
# Сегменты для среднего времени сдачи экзамена
def r_score(x):
    if x <= quantiles['average_pass_date'][.33]:
        return 1
    elif x <= quantiles['average_pass_date'][.66]:
        return 2
    else:
        return 3


# Сегменты для завершаемости курсов(принимаем доп условие по равенству квантилей, если они равны,
# то задаем границы как 0.33 и 0.66)
def f_score(x):
    if (
        quantiles['passing_rate'][0.33] != quantiles['passing_rate'][0.66]
        and quantiles['passing_rate'][0.66] != RFM.passing_rate.max()
    ):
        if x <= quantiles['passing_rate'][0.33]:
            return 3
        elif x <= quantiles['passing_rate'][0.66]:
            return 2
        else:
            return 1
    else:
        if x == 0:
            return 3
        elif x == 0.5:
            return 2
        else:
            return 1


# Сегменты для среднего количества баллов:
def m_score(x):
    if x <= quantiles['average_score'][0.33]:
        return 3
    elif x <= quantiles['average_score'][.66]:
        return 2
    else:
        return 1


# Определяем значения столбцов RFM-кластеризации:
RFM['R'] = RFM['average_pass_date'].apply(lambda x: r_score(x))
RFM['F'] = RFM['passing_rate'].apply(lambda x: f_score(x))
RFM['M'] = RFM['average_score'].apply(lambda x: m_score(x))

In [32]:
# Определим, в каких группах находятся студенты:
RFM[['R', 'F', 'M']] = RFM[['R', 'F', 'M']].astype(str)

RFM['cluster'] = RFM['R'] + RFM['F'] + RFM['M']

In [33]:
# сгруппируем студентов по кластерам для вывода:
grouped_RFM = RFM.groupby('cluster', as_index=False) \
                 .agg({'average_pass_date': 'count'}) \
                 .rename(columns={'average_pass_date': 'number_of_students'}) \
                 .sort_values('number_of_students', ascending=False)

In [34]:
# Напишем функцию для вывода значений границ сегментов:
def Cluster_analysis(df):
    """Gets dataframe with 2 columns: cluster and number of elements in cluster
       Print analysis of cluster with borders and number of elements"""
    print(f'Число ненулевых кластеров: {df.shape[0]}\nRFM-сегменты представлены в виде следующих групп:\n')
#     Проходтим по таблице через цикл с i для извлечения группы и числа студентов в ней
    for i in range(0, df.shape[0] - 1):
        cluster = df.iloc[i, 0]
        number = df.iloc[i, 1]
        if cluster[0] == '1':
            average_pass_date = f"от {RFM.average_pass_date.min()} до {quantiles['average_pass_date'][0.33]} дней"
        elif cluster[0] == '2': 
            average_pass_date = (f"от {quantiles['average_pass_date'][0.33]} до " 
                                 f"{quantiles['average_pass_date'][0.66]} дней")
        else:
            average_pass_date = (f"от {quantiles['average_pass_date'][0.66]} до "
                                 f"{RFM.average_pass_date.max()} дней")
        if cluster[1] == '3':
            passing_rate = f"от {RFM.passing_rate.min()} до 0.33"
        elif cluster[1] == '2': 
            passing_rate = "от 0.33 до 0.66"
        else:
            passing_rate = f"от 0.66 до {RFM.passing_rate.max()}"
        if cluster[2] == '3':
            average_score = f"от {RFM.average_score.min()} до {quantiles['average_score'][0.33]} баллов"
        elif cluster[2] == '2': 
            average_score = f"от {quantiles['average_score'][0.33]} до {quantiles['average_score'][0.66]} баллов"
        else:
            average_score = f"от {quantiles['average_score'][0.66]} до {RFM.average_score.max()} баллов"
        print(f'{cluster} (число студентов:{number}) имеет границы метрик average_pass_date {average_pass_date}, '
              f'passing_rate {passing_rate}, average_score {average_score}')

In [35]:
Cluster_analysis(grouped_RFM)

Число ненулевых кластеров: 16
RFM-сегменты представлены в виде следующих групп:

212 (число студентов:693) имеет границы метрик average_pass_date от 236.0 до 243.0 дней, passing_rate от 0.66 до 1.0, average_score от 56.0 до 76.0 баллов
211 (число студентов:586) имеет границы метрик average_pass_date от 236.0 до 243.0 дней, passing_rate от 0.66 до 1.0, average_score от 76.0 до 100.0 баллов
111 (число студентов:546) имеет границы метрик average_pass_date от 229.0 до 236.0 дней, passing_rate от 0.66 до 1.0, average_score от 76.0 до 100.0 баллов
112 (число студентов:541) имеет границы метрик average_pass_date от 229.0 до 236.0 дней, passing_rate от 0.66 до 1.0, average_score от 56.0 до 76.0 баллов
213 (число студентов:473) имеет границы метрик average_pass_date от 236.0 до 243.0 дней, passing_rate от 0.66 до 1.0, average_score от 0.0 до 56.0 баллов
113 (число студентов:408) имеет границы метрик average_pass_date от 229.0 до 236.0 дней, passing_rate от 0.66 до 1.0, average_score от 0.0 до 5

Таким образом, большая часть студентов успешно справляется с экзаменами на курсе (сегменты с лучшими показателями средних значений
сроков сдачи экзаменов и баллов с завершаемостью самые наполненные), однако в данном случае использование метрики passing_rate выглядит
не информативно, т.к. большинство студентов проходили 1 курс и сегментация по данной метрике задана искусственными границами между 
тремя величинами: 0, 0.5 и 1.0, поэтому я бы указывал вместо границ passing_rate действительное значение.
При наличии большого количества величин в колонке passing_rate, задание границ будет обоснованным